In [1]:
# GOAL: get a L2 cutout date and time at low longitude and high longitude
# see what date it is extracting from L3

import xarray
import numpy as np
import pandas
import h5py
import datetime

In [2]:
l3s = pandas.read_parquet('/Users/rachelstumpf/SST_L3S_VIIRS.parquet')
gd_date = l3s.datetime <= datetime.datetime(2012,2,2)
df = l3s[gd_date]

In [3]:
df[['lon', 'VIIRS_datetime', 'VIIRS_UID']].sort_values(by=['lon'])

,lon,VIIRS_datetime,VIIRS_UID
941635,-179.99990,2014-12-31 13:30:00,1420129744910
2154637,-179.99986,2018-09-25 01:20:00,153793412114
500259,-179.99979,2013-08-07 13:30:00,137550022021
1582103,-179.99974,2017-01-10 02:30:00,148455197326
2145523,-179.99960,2018-09-14 12:40:00,153651655440
...,...,...,...
2877804,179.99928,2020-10-26 01:20:00,160350264035999928
1325407,179.99947,2016-03-25 01:50:00,145837662335999947
1833393,179.99957,2017-10-03 12:30:00,150749579835999957
632803,179.99991,2014-01-08 13:30:00,1389110817635999991


In [4]:
# okay we're going to look at 
# CUTOUT 1: VIIRS_UID = 1328126105416441, lon = 179.83559, index = 387
# CUTOUT 2: VIIRS_UID = 1328102862635982285, lon = 179.82285, index = 27

cutout_1 = df[(df['VIIRS_UID'] == 1328126105416441)]
cutout_2 = df[(df['VIIRS_UID'] == 1328102862635982285)]

In [5]:
# cutout 1 and time conversions

cutout_1['VIIRS_datetime']

#base_datetime = pandas.to_datetime(cutout_1['VIIRS_datetime']).dt.date.astype(str) + ' 01:30:00'

 #   base_datetime = pandas.to_datetime(base_datetime, format='%Y-%m-%d %H:%M:%S')
 #   l3s['datetime'] = (base_datetime + pandas.to_timedelta(l3s['VIIRS_lon'] * 4, unit='minutes')).dt.round('S')

387   2012-02-01 14:20:00
Name: VIIRS_datetime, dtype: datetime64[ns]

In [6]:
# take the VIIRS date, convert it to L3S date,
# you selected it based on the VIIRS date anyway, 
# what you need to do is select on closest L3S date

base_datetime = pandas.to_datetime(cutout_1['VIIRS_datetime']).dt.date.astype(str) + ' 01:30:00'
base_datetime = pandas.to_datetime(base_datetime, format='%Y-%m-%d %H:%M:%S')
given_date = (base_datetime + pandas.to_timedelta(cutout_1['VIIRS_lon'] * 4, unit='minutes')).dt.round('S')

In [7]:
# take this given date: 
# If it is before 12:00:00 on the original VIIRS date, we want to take the L3S cutout from that VIIRS date
# If it is after 12:00:00 on the previous day's VIIRS date, we want to take the L3S cutout from that VIIRS date
# If it is before 12:00:00 on the previous day's VIIRS date, we want to take the L3S cutout from the previous VIIRS date
# If it is after 12:00:00 on the original VIIRS date, we want to take the L3S cutout from the next VIIRS date

given_date = given_date.dt.date
viirs_date = cutout_1['VIIRS_datetime'].dt.date

In [8]:
viirs_date = cutout_1['VIIRS_datetime'].dt.date
desired_time = datetime.time(1, 30, 0)
combined_datetime = datetime.datetime.combine(viirs_date.iloc[0], desired_time)

datetime = (combined_datetime + pandas.to_timedelta(cutout_1['VIIRS_lon'] * 4, unit='minutes')).round('S')
datetime

387   2012-01-31 13:30:39
Name: VIIRS_lon, dtype: datetime64[ns]

In [9]:
# viirs_date = datetime.datetime.combine(cutout_1['VIIRS_datetime'].dt.date.iloc[0], desired_time)
# viirs_date_minus1 = datetime.datetime.combine(cutout_1['VIIRS_datetime'].dt.date.iloc[0] - datetime.timedelta(days=1), desired_time)
# viirs_date_plus1 = datetime.datetime.combine(cutout_1['VIIRS_datetime'].dt.date.iloc[0] + datetime.timedelta(days=1), desired_time)

In [10]:
# re-run extractions to see if anything with data size has changed?
# it appears that even when a cutout is very low or very high longitude, it is not enough to shift it more than
# 12 hours away from the 1:30 am time so they are usually just on the same day as the VIIRS cutout